In [1]:
from mpmath import mp

In [2]:

def extract_pi_sequence(start_pos=0, end_pos=1000):
    """
    Extract a sequence of Pi digits between two base-pair numbers.

    Args:
    start_pos (int): The starting position of the Pi sequence (1-indexed).
    end_pos (int): The ending position of the Pi sequence (1-indexed).

    Returns:
    str: The extracted Pi sequence.
    """
    # Set the precision (number of digits) for Pi calculation
    mp.dps = end_pos + 10  # Set slightly higher than end_pos for safety

    # Get the Pi value as a string
    pi_str = str(mp.pi)[2:]  # Remove the "3."

    # Extract the sequence between start_pos and end_pos
    pi_sequence = pi_str[start_pos - 1:end_pos]  # Adjust for 0-indexing

    return pi_sequence


In [3]:

def extract_and_convert_pi_to_binary(start_pos, end_pos):
    """
    Extract a sequence of Pi digits between two base-pair numbers and convert the sequence into binary.

    Args:
    start_pos (int): The starting position of the Pi sequence (1-indexed).
    end_pos (int): The ending position of the Pi sequence (1-indexed).

    Returns:
    str: The binary representation of the extracted Pi sequence.
    """
    # Extract the Pi sequence
    #pi_sequence = extract_pi_sequence(start_pos, end_pos)
    pi_sequence = get_pi_sequence()

    # Convert each digit in the sequence to binary and concatenate the result
    binary_sequence = ''.join(format(int(digit), '04b') for digit in pi_sequence)

    return binary_sequence


In [4]:

def binary_to_text(binary_sequence):
    """
    Convert a binary string to human-readable text.

    Args:
    binary_sequence (str): A string of binary numbers (e.g., '01001000011001010110110001101100').

    Convert Binary to ASCII:
    Each 8-bit binary chunk is converted to its decimal equivalent using int(byte, 2), and then to an ASCII character using chr().
    text = ''.join([chr(int(byte, 2)) for byte in bytes_list]) joins all the characters into a single string.

    Returns:
    str: The corresponding human-readable text.
    """
    # Split the binary string into chunks of 8 bits (1 byte)
    bytes_list = [binary_sequence[i:i + 8] for i in range(0, len(binary_sequence), 8)]

    # Convert each byte to its ASCII character
    text = ''.join([chr(int(byte, 2)) for byte in bytes_list])

    return text


In [5]:

def introns(pi_sequence, intron_ranges):
    """
    Remove the segments defined as introns from the Pi sequence.

    Args:
    pi_sequence (str): The original Pi sequence.
    intron_ranges (list of tuples): List of (start, end) tuples defining the intron segments (1-indexed).

    Returns:
    str: The Pi sequence with introns removed.
    """
    intron_free_sequence = pi_sequence
    for start, end in sorted(intron_ranges, reverse=True):
        intron_free_sequence = intron_free_sequence[:start - 1] + intron_free_sequence[end:]
    return intron_free_sequence


In [6]:

def exons(pi_sequence, intron_ranges):
    """
    Extract the segments defined as exons from the Pi sequence (i.e., segments not defined as introns).

    Args:
    pi_sequence (str): The original Pi sequence.
    intron_ranges (list of tuples): List of (start, end) tuples defining the intron segments (1-indexed).

    Returns:
    list: A list of exon segments as strings.
    """
    exons_list = []
    last_end = 0
    for start, end in sorted(intron_ranges):
        if start > last_end + 1:
            exons_list.append(pi_sequence[last_end:start - 1])
        last_end = end
    if last_end < len(pi_sequence):
        exons_list.append(pi_sequence[last_end:])
    return exons_list


In [7]:

def spliceosome(exons_list):
    """
    Combine the exons into a single spliced sequence.

    Args:
    exons_list (list): List of exon segments as strings.

    Returns:
    str: The spliced Pi sequence.
    """
    return ''.join(exons_list)


In [8]:

def find_word_in_pi(word, start_pos=3, pi_length=1000):
    """
    Search for a human-readable word in the Pi sequence and return its start and end positions.

    Args:
    word (str): The word to search for in the Pi sequence.
    start_pos (int): The starting position to begin the search in the Pi sequence (0-indexed).

    Returns:
    tuple: A tuple containing the start and end positions of the word in the Pi sequence, or None if not found.
    """
    # Convert the word to its binary representation
    binary_word = ''.join(format(ord(char), '08b') for char in word)

    # Set the precision to cover a large part of the Pi sequence
    mp.dps = pi_length  # Adjust based on the range needed

    # Get the Pi value as a string
    pi_str = str(mp.pi)[2:]  # Remove the "3."

    # Convert the Pi sequence to binary
    binary_pi_sequence = ''.join(format(int(digit), '04b') for digit in pi_str[start_pos:])

    # Find the binary word in the binary Pi sequence
    start_index = binary_pi_sequence.find(binary_word)

    if start_index != -1:
        # Calculate the start and end positions in terms of digits in the Pi sequence
        start_position = start_index // 4 + start_pos + 1
        end_position = start_position + len(word) * 2 - 1
        return start_position, end_position
    else:
        return None


In [9]:

def get_pi_sequence():
    
    # Read the contents of the pi.out file
    with open("pi.out", "r") as file:
        pi_digits = file.read()
    return pi_digits


In [10]:

# Example usage:
start_pos = 333
end_pos = 999

# Define the intron ranges (1-indexed, inclusive)
intron_ranges = [(100, 200), (300, 400), (600, 700)] #control
intron_ranges = [(50, 200), (35, 400), (678, 789), (78, 189)]

# Extract the Pi sequence
pi_sequence = extract_pi_sequence(start_pos, end_pos)
pi_sequence = get_pi_sequence()

print(f"\nThe Pi sequence:\n{pi_sequence}")

print(f"\nThe Pi sequence in binary:\n{extract_and_convert_pi_to_binary(start_pos,end_pos)}")

# Remove introns from the pi_sequence
intron_free_sequence = introns(pi_sequence, intron_ranges)
print(f"\nIntron-free Pi sequence:\n{intron_free_sequence}")



The Pi sequence:
3141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724587006606315588174881520920962829254091715364367892590360011330530548820466521384146951941511609433057270365759591953092186117381932611793105118548074462379962749567351885752724891227938183011949129833673362440656643086021394946395224737190702179860943702770539217176293176752384674818467669405132000568127145263560827785771342757789609173637178721468440901224953430146549585371050792279689258923542019956112129021960864034418159813629774771309960518707211349999998372978049951059731732816096318595024459455346908302642522308253344685035261931188171010003137838752886587533208381420617177669147303598253490428755468731159562863882353787593751957781857780532171226806613001927876

In [11]:

# Extract exons
exons_list = exons(pi_sequence, intron_ranges)
print(f"\nExons extracted:\n{exons_list}")

# Combine exons to create the spliced sequence
spliced_sequence = spliceosome(exons_list)
print(f"\nSpliced Pi sequence:\n{spliced_sequence}")

# Convert the spliced Pi sequence to binary
binary_spliced_sequence = ''.join(format(int(digit), '04b') for digit in spliced_sequence)

# Convert the binary sequence to human-readable text
human_text = binary_to_text(binary_spliced_sequence)
print(f"\nThe human-readable text from the spliced sequence is:\n{human_text}")



Exons extracted:
['3141592653589793238462643383279502', '89549303819644288109756659334461284756482337867831652712019091456485669234603486104543266482133936072602491412737245870066063155881748815209209628292540917153643678925903600113305305488204665213841469519415116094330572703657595919530921861173819326117931051185480744623799627495673518857527248912279381830119491298336733624406566430860213949463952247371907021798609437027705392171762931767523846748184676694051320005681271452635608277857713427577896091736371787214684409012249534301465495', '2816096318595024459455346908302642522308253344685035261931188171010003137838752886587533208381420617177669147303598253490428755468731159562863882353787593751957781857780532171226806613001927876611195909216420198']

Spliced Pi sequence:
3141592653589793238462643383279502895493038196442881097566593344612847564823378678316527120190914564856692346034861045432664821339360726024914127372458700660631558817488152092096282925409171536436789259036001133053